In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import diffusionstuff7 as ds
from copy import copy as dup
from scipy.integrate import odeint
from matplotlib import rcParams
from sim_handling import multiple_test_avg_time, Simulation

C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [2]:
#Meta testing parameters
number_of_tests = 10

In [3]:
# Graphics parameters
%matplotlib widget
ticklabelsize = 15
fontsize = 15
linewidth = 2
fignum = 0

In [4]:
zero_d_test = Simulation(ds.f0d,shape=(1,),rtol=1e-12)
multiple_test_avg_time(zero_d_test.results, n_tests=number_of_tests)

TypeError: No matching definition for argument type(s) array(float64, 1d, C), float64, float64, int32

: 